In [0]:
#Create Sample Dataframe

array_data = [("John",4,1),
              ("John",6,2),
              ("David",7,3),
              ("Dave",5,2),
              ("John",7,3),
              ("John",9,7),
              ("David",1,8),
              ("David",4,9),
              ("David",7,4),
              ("Mike",5,2),
              ("Mike",3,5),
              ("John",2,7),
              ("Davie",1,9)
             ]
array_schema = ["Name","Score_1","Score_2"]
arrayDF = spark.createDataFrame(data =array_data, schema = array_schema)
display(arrayDF)

Name,Score_1,Score_2
John,4,1
John,6,2
David,7,3
Dave,5,2
John,7,3
John,9,7
David,1,8
David,4,9
David,7,4
Mike,5,2


In [0]:
#Convert Sample Dataframe into Array Dataframe
from pyspark.sql import functions as F

masterDF = arrayDF.groupby("Name").agg(F.collect_list("Score_1").alias('Array_score_1'),F.collect_list('Score_2').alias('Array_score_2'))

display(masterDF)
masterDF.printSchema()

Name,Array_score_1,Array_score_2
John,"List(4, 6, 7, 9, 2)","List(1, 2, 3, 7, 7)"
David,"List(7, 1, 4, 7)","List(3, 8, 9, 4)"
Dave,List(5),List(2)
Mike,"List(5, 3)","List(2, 5)"
Davie,List(1),List(9)


root
 |-- Name: string (nullable = true)
 |-- Array_score_1: array (nullable = false)
 |    |-- element: long (containsNull = false)
 |-- Array_score_2: array (nullable = false)
 |    |-- element: long (containsNull = false)



In [0]:
#Apply arrays_zip function on Array DF
arr_zip_df = masterDF.withColumn("Zipped_value", F.arrays_zip("Array_Score_1", "Array_Score_2"))
arr_zip_df.show(10, False)

+-----+---------------+---------------+----------------------------------------+
|Name |Array_score_1  |Array_score_2  |Zipped_value                            |
+-----+---------------+---------------+----------------------------------------+
|John |[4, 6, 7, 9, 2]|[1, 2, 3, 7, 7]|[{4, 1}, {6, 2}, {7, 3}, {9, 7}, {2, 7}]|
|David|[7, 1, 4, 7]   |[3, 8, 9, 4]   |[{7, 3}, {1, 8}, {4, 9}, {7, 4}]        |
|Dave |[5]            |[2]            |[{5, 2}]                                |
|Mike |[5, 3]         |[2, 5]         |[{5, 2}, {3, 5}]                        |
|Davie|[1]            |[9]            |[{1, 9}]                                |
+-----+---------------+---------------+----------------------------------------+



In [0]:
#Practical Use Case to flatten data using arrays_zip and explode

empDF = [
('Sales_dept',[{'emp_name':'John','salary':'1000','yrs_of_service':'18','Age':'33'},
               {'emp_name':'David','salary':'2000','yrs_of_service':'15','Age':'48'},
               {'emp_name':'Nancy','salary':'4000','yrs_of_service':'29','Age':'45'},
               {'emp_name':'Mike','salary': '3000','yrs_of_service':'6','Age':'20'},
               {'emp_name':'Rosy','salary':'7000','yrs_of_service':'3','Age':'32'}]),

('HR_dept',[{'emp_name': 'Edvin','salary':'6000','yrs_of_service': '8','Age':'31'},
            {'emp_name':'Tonas', 'salary':'5000','yrs_of_service': '4','Age':'26'},
            {'emp_name':'Sarah', 'salary':'12000','yrs_of_service':'22','Age': '48'},
            {'emp_name': 'Stella','salary':'15000','yrs_of_service':'25','Age':'52'},
             {'emp_name':'Kevin','salary':'4000', 'yrs_of_service': '5','Age':'27'}])
]

df_brand = spark.createDataFrame(data=empDF,schema=['Department', 'Employee'])
df_brand.printSchema()
display(df_brand)


root
 |-- Department: string (nullable = true)
 |-- Employee: array (nullable = true)
 |    |-- element: map (containsNull = true)
 |    |    |-- key: string
 |    |    |-- value: string (valueContainsNull = true)



Department,Employee
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))"
HR_dept,"List(Map(emp_name -> Edvin, yrs_of_service -> 8, salary -> 6000, Age -> 31), Map(emp_name -> Tonas, yrs_of_service -> 4, salary -> 5000, Age -> 26), Map(emp_name -> Sarah, yrs_of_service -> 22, salary -> 12000, Age -> 48), Map(emp_name -> Stella, yrs_of_service -> 25, salary -> 15000, Age -> 52), Map(emp_name -> Kevin, yrs_of_service -> 5, salary -> 4000, Age -> 27))"


In [0]:
#Apply Arrays_zip
df_brandZip = df_brand.withColumn("Zip",F.arrays_zip(df_brand["Employee"]))
display(df_brandZip)

Department,Employee,Zip
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48)), List(Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20)), List(Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32)))"
HR_dept,"List(Map(emp_name -> Edvin, yrs_of_service -> 8, salary -> 6000, Age -> 31), Map(emp_name -> Tonas, yrs_of_service -> 4, salary -> 5000, Age -> 26), Map(emp_name -> Sarah, yrs_of_service -> 22, salary -> 12000, Age -> 48), Map(emp_name -> Stella, yrs_of_service -> 25, salary -> 15000, Age -> 52), Map(emp_name -> Kevin, yrs_of_service -> 5, salary -> 4000, Age -> 27))","List(List(Map(emp_name -> Edvin, yrs_of_service -> 8, salary -> 6000, Age -> 31)), List(Map(emp_name -> Tonas, yrs_of_service -> 4, salary -> 5000, Age -> 26)), List(Map(emp_name -> Sarah, yrs_of_service -> 22, salary -> 12000, Age -> 48)), List(Map(emp_name -> Stella, yrs_of_service -> 25, salary -> 15000, Age -> 52)), List(Map(emp_name -> Kevin, yrs_of_service -> 5, salary -> 4000, Age -> 27)))"


In [0]:
#Apply Explode

df_brand_exp = df_brandZip.withColumn("Explode", F.explode(df_brandZip.Zip))
display(df_brand_exp)

Department,Employee,Zip,Explode
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48)), List(Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20)), List(Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32)))","List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48)), List(Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20)), List(Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32)))","List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48)), List(Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20)), List(Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32)))","List(Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48)), List(Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45)), List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20)), List(Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32)))","List(Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20))"
Sales_dept,"List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33), Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48), Map(emp_name -> Nancy, yrs_of_service -> 29, salary -> 4000, Age -> 45), Map(emp_name -> Mike, yrs_of_service -> 6, salary -> 3000, Age -> 20), Map(emp_name -> Rosy, yrs_of_service -> 3, salary -> 7000, Age -> 32))","List(List(Map(emp_name -> John, yrs_of_service -> 18, salary -> 1000, Age -> 33)), List(Map(emp_name -> David, yrs_of_service -> 15, salary -> 2000, Age -> 48)), List(Map(e

In [0]:
#Flatten fields from exploded list
df_brand_output = df_brand_exp.withColumn("employee_emp_name",df_brand_exp['Explode.Employee.emp_name'])\
                                .withColumn("employee_yrs_of_service",df_brand_exp['Explode.Employee.yrs_of_service'])\
                                .withColumn("employee_salary",df_brand_exp['Explode.Employee.salary'])\
                         .withColumn("employee_Age",df_brand_exp['Explode.Employee.Age']).drop("Explode").drop("Zip").drop("Employee")
display(df_brand_output)

Department,employee_emp_name,employee_yrs_of_service,employee_salary,employee_Age
Sales_dept,John,18,1000,33
Sales_dept,David,15,2000,48
Sales_dept,Nancy,29,4000,45
Sales_dept,Mike,6,3000,20
Sales_dept,Rosy,3,7000,32
HR_dept,Edvin,8,6000,31
HR_dept,Tonas,4,5000,26
HR_dept,Sarah,22,12000,48
HR_dept,Stella,25,15000,52
HR_dept,Kevin,5,4000,27
